# LangChain Chains

- Agora que aprendemos sobre entradas e saídas de modelos e conexões de dados, podemos finalmente aprender sobre chains.
- Chains nos permite vincular a saída de uma chamada LLM à entrada de outra chamada.
- Langchain também fornece muitas funcionalidades de chains integradas, como encadear pesquisas de similaridade de documentos com outras chamadas LLM, ações que construímos anteriormente manualmente com Langchain.

- https://python.langchain.com/docs/modules/chains/foundational/

- Chains tem um bloco de construção básico conhecido como objeto LLMChain.
- Você pode pensar no LLMChain apenas como uma simples chamada LLM que terá uma entrada e uma saída.

In [ ]:
from dotenv import load_dotenv
load_dotenv()
import os

from langchain.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

In [ ]:
chat_llm = AzureChatOpenAI(
    openai_api_type=os.getenv('AZOPENAI_API_TYPE'),
    openai_api_key=os.getenv('AZOPENAI_API_KEY'),
    openai_api_base=os.getenv('AZOPENAI_API_BASE'),
    openai_api_version=os.getenv('AZOPENAI_DEPLOYMENT_VERSION'),
    deployment_name=os.getenv('AZOPENAI_DEPLOYMENT_NAME'),
    model=os.getenv('AZOPENAI_MODEL_NAME'),
)


In [ ]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate

In [ ]:
human_prompt = "Descreva as atribuições do cargo de {nome_cargo}"
human_prompt_template = HumanMessagePromptTemplate.from_template(human_prompt)
chat_prompt_template = ChatPromptTemplate.from_messages([human_prompt_template])

In [ ]:
# Antes
prompt = chat_prompt_template.format_messages(nome_cargo='Auditor de Controle Externo')
chat_llm(prompt)

In [ ]:
# Depois
# Large Language Model Chain
from langchain.chains import LLMChain

chain = LLMChain(llm=chat_llm,  prompt=chat_prompt_template)
chain

In [ ]:
result = chain.run(nome_cargo='Auditor de Controle Externo')
result # O retorno é uma String!

## Simple Sequential Chain
- Agora que entendemos como usar o objeto LLMChain, podemos encadeá-los para criar funcionalidades mais complexas com Langchain.
- Lembre-se que um processo de engenharia de prompt pode necessitar de várias etapas, podemos automatizar essas etapas.
- *input -> LLMChain -> LLMChain -> LLMChain -> output*

In [ ]:
from langchain.chains import SimpleSequentialChain
# Um INPUT e um OUTPUT para cada CHAIN

In [ ]:
# Escrita de um documento

# Definindo os tópicos/etapas
prompt_1 = 'Você é um especialista em auditorias usando a NBASP, descreva as etapas como uma lista numérica para um processo de auditoria de {area_auditoria}. Lembrese de descrever somente as etapas.'
# Notem que o HumanPrompt foi criado automaticamente
prompt_template_1 = ChatPromptTemplate.from_template(prompt_1)
chain_topicos = LLMChain(llm=chat_llm, prompt=prompt_template_1)

In [ ]:
prompt_2 = 'Você é um especialista em auditorias usando a NBASP, escreva os detalhes das as etapas descritas a seguir: {lista_etapas}'
prompt_template_2 = ChatPromptTemplate.from_template(prompt_2)
# Poderiamos mudar o modelo
chain_detalhes = LLMChain(llm=chat_llm, prompt=prompt_template_2)

In [ ]:
# A ordem importa, a segunda variável (lista_etapas) é identificada automaticamente
full_chain = SimpleSequentialChain(chains=[chain_topicos, chain_detalhes], verbose=True)

In [ ]:
resultado = full_chain.run('Transferências Volutárias')

In [ ]:
print(resultado)

### Sequential Chain
- SequentialChains são muito semelhantes aos SimpleSequentialChains, mas nos permitem ter acesso a todas as saídas dos LLMChains internos.


In [ ]:
from langchain.chains import SequentialChain

In [ ]:
chain_topicos = LLMChain(llm=chat_llm, prompt=prompt_template_1, output_key='lista_etapas')
chain_detalhes = LLMChain(llm=chat_llm, prompt=prompt_template_2, output_key='texto_final')

prompt_3 = 'Escreva uma notícia curta para ser publicada convidando os auditores a ler o texto abaixo: \n{texto_final}'
prompt_template_3 = ChatPromptTemplate.from_template(prompt_3)
chain_noticia = LLMChain(llm=chat_llm, prompt=prompt_template_3, output_key='noticia')

In [ ]:
seq_chain = SequentialChain(
    chains=[chain_topicos, chain_detalhes, chain_noticia],
    input_variables=['area_auditoria'],
    output_variables=['lista_etapas', 'texto_final', 'noticia'],
    verbose=True
)

In [ ]:
resultado = seq_chain("Auditoria de Obras")
# Os resultados não serão mostrados, pois vc tem acesso a todas as saídas

In [ ]:
from pprint import pprint
pprint(resultado)